In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
!unzip '/kaggle/input/spooky-author-identification/train.zip'


In [ ]:
!unzip '/kaggle/input/spooky-author-identification/test.zip'
!unzip '/kaggle/input/spooky-author-identification/sample_submission.zip'


In [ ]:
data = pd.read_csv('train.csv')
data.head()


In [ ]:
test = pd.read_csv('test.csv')
test.head()


In [ ]:
sample = pd.read_csv('sample_submission.csv')
sample.head()


In [ ]:
data.shape


In [ ]:
import matplotlib.pyplot as plt

data['author'].value_counts(normalize=True).plot(kind='bar')
plt.show()


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

# 데이터 로드 및 준비
def load_data(file_path):
    df = pd.read_csv(file_path)
    texts = df['text'].tolist()
    authors = df['author'].tolist()
    return texts, authors

def load_test_data(file_path):
    df = pd.read_csv(file_path)
    texts = df['text'].tolist()
    ids = df['id'].tolist()
    return texts, ids

file_path = "train.csv"  # 학습 데이터 경로
test_file_path = "test.csv"  # 테스트 데이터 경로

texts, authors = load_data(file_path)
test_texts, test_ids = load_test_data(test_file_path)

# 텍스트 벡터화
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(texts)

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(authors)

# 데이터셋 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 나이브 베이즈 모델 정의 및 학습
model = MultinomialNB()
model.fit(X_train, y_train)

# 검증 데이터 성능 평가
y_val_pred = model.predict(X_val)
print("Validation Classification Report:\n")
print(classification_report(y_val, y_val_pred, target_names=label_encoder.classes_))

# 테스트 데이터에 대한 예측
test_X = vectorizer.transform(test_texts)
test_predictions = model.predict_proba(test_X)

# 결과 저장
result_df = pd.DataFrame(test_predictions, columns=label_encoder.classes_)
result_df['id'] = test_ids
result_df
result_df.to_csv("test_predictions.csv", index=False)
